# **Réalisez un traitement dans un environnement Big Data sur le Cloud**

## **Import des librairies**

In [1]:
# Mesure des durées d'exécution : 
import time
# Pour lire les images qui auront été chargées au format binaire : 
import io
# Pour la gestion des chemins de fichiers
import os

# Pour valider les résultats et les exporter en CSV
import pandas as pd
# Pour redimensionner les images
from PIL import Image
# Pour la manipulation d'arrays
import numpy as np

# Désactivation des messages de debugging de tensorflow.
# Doit être exécuté avant les imports de tensorflow.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Pour l'extraction de features des images
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

# Création et manipulation de dataframes Spark
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

# Pour convertion des features en vecteurs Spark
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
# Pour réalisation de la PCA sur les features
from pyspark.ml.feature import PCA

### **Mesure de la durée d'exécution du notebook**

In [2]:
start_time = time.perf_counter()

## **1. Définition des PATH pour charger les images et enregistrer les résultats**

In [3]:
PATH = os.getcwd()  # Dossier courant
PATH_Data = PATH + '/data/Test1'
PATH_Result = PATH + '/data/Results'
print('PATH :        ' + PATH\
      + '\nPATH_Data :   ' + PATH_Data \
      + '\nPATH_Result : ' + PATH_Result)

PATH :        /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers
PATH_Data :   /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Test1
PATH_Result : /mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Results


## **2. Création de la SparkSession**

In [4]:
spark = (SparkSession
             .builder
             .appName('P8')  # Nom de l'application
             .master('local')  # Pour une exécution en local, local[x] si on veut utiliser x cœurs (sinon tous les dispos sont utilisés)
             .config("spark.sql.parquet.writeLegacyFormat", 'true')  # Pour pouvoir utiliser le format parquet lors de l'enregistrement des résultats
             .getOrCreate()  # obtenir une session spark existante ou si aucune n'existe, en créer une nouvelle
)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Nous créons également la variable "**sc**" qui est un **SparkContext** issue de la variable **spark** :

In [5]:
sc = spark.sparkContext

Affichage des informations de Spark en cours d'execution :

In [6]:
spark

## **3. Traitement des données**

### **3.1 Chargement des données**

Les images sont chargées au format binaire, ce qui offre plus de souplesse dans la façon de prétraiter les images.

Avant de charger les images, nous spécifions que nous voulons charger uniquement les fichiers dont l'extension est **jpg**.

Nous indiquons également de charger tous les objets possibles contenus dans les sous-dossiers du dossier communiqué ("recursiveFileLookup").

In [7]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

Ajout d'une colonne contenant les **labels** de chaque image :

In [8]:
# split(images['path'], '/'),-2) : couper le chemin complet à chaque '/'
# et prendre l'avant dernier élément, c'est-à-dire le nom du dossier
# qui contient les images
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())  # Affichage de la structure du dataframe
print(images.select('path','label').show(5,True)) # False si on ne veut pas tronquer les colonnes lors de l'affichage

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------+------------------+
|                path|             label|
+--------------------+------------------+
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
|file:/mnt/c/Users...|Apple Crimson Snow|
+--------------------+------------------+
only showing top 5 rows

None


### **3.2 Préparation du modèle**

Nous allons utiliser la technique du **transfert learning** pour extraire les features des images.

Nous allons utiliser le modèle **MobileNetV2** pour sa rapidité d'exécution comparée à d'autres modèles comme *VGG16* par exemple. Nous allons donc récupérer l'avant dernière couche du modèle en sortie. La dernière couche, avec sa fonction d'activation *Softmax*, est destinée à la classification, ce que nous ne souhaitons pas ici.

**MobileNetV2**, lorsqu'on l'utilise en incluant toutes ses couches, attend obligatoirement des images de dimension (224,224,3). Nos images étant toutes de dimension (100,100,3), nous devrons simplement les **redimensionner** avant de les confier au modèle.

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec :
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

Affichage du résumé de notre nouveau modèle où nous constatons que <u>nous récupérons en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [11]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser ensuite les poids aux différents workeurs.

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

<u>Mettons cela sous forme de fonction</u> :

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)  # Diffusion des poids
    return new_model

### **3.3 Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF**

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    # Chargement des images au format binaire et redimensionnement
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)  # Conversion en array
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # Pour certaines couches, les caractéristiques de sortie seront des tenseurs multidimensionnels.
    # Nous aplatissions les tenseurs de caractéristiques en vecteurs pour faciliter le stockage dans les DataFrames Spark.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # Avec PandasUDFType.SCALAR_ITER, nous pouvons charger le modèle une fois puis le réutiliser
    # pour plusieurs lots de données. Cela amortit les coûts liés au chargement des modèles volumineux.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/home/maxime/.local/lib/python3.10/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


### **3.4 Exécution des actions d'extraction de features**

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), peuvent rencontrer des erreurs de type Out Of Memory (OOM).

Si de telles erreurs arrivent ci-dessous, il est possible de réduire la taille du lot Arrow via 'maxRecordsPerBatch'

Il ne devrait pas y avoir de problème ici, je laisse donc la commande en commentaire.

In [15]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

Il s'agit ici d'une étape de transformation dans Spark, autrement dit l'extraction réelle des features n'aura pas encore lieu, elle sera déclenchée par une action plus tard.

In [16]:
# Choix du nombre de partitions que l'on va créer avec "images"
nb_partitions = 20

features_df = images.repartition(nb_partitions).select(col("path"),
                                                       col("label"),
                                                       featurize_udf("content").alias("features")
                                                      )

Affichage de la structure du dataframe *features_df* : 

In [17]:
features_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)



### **3.5 Réalisation de la PCA**

Conversion de la colonne features en vecteurs, car c'est le format d'entrée requis pour faire une PCA avec Spark : 

In [18]:
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
features_df = features_df.withColumn('features', list_to_vector_udf('features'))

On choisit le nombre de composantes à garder avec la PCA : 

In [19]:
n_componants = 20

Réalisation de la PCA, ce qui va constituer une action qui va déclencher les calculs de featurisation.

En effet, pour réaliser la PCA, l'ensemble des features doit être disponible pour pouvoir calculer la matrice de covariance, les vecteurs propres et les valeurs propres associées.

Par conséquent, on ne peut pas effectuer une PCA au fur et à mesure que les valeurs sont créées.

In [20]:
pca = PCA(k=n_componants, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(features_df)
features_df = model.transform(features_df).select("path", "label", "features", "pcaFeatures")

1/1 [==============================] - 2s 2s/step                   (0 + 1) / 1]
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
1/1 [==============================] - 0s 477ms/step===========>  (19 + 1) / 20]


A-t-on choisi un nombre adéquat de composantes pour notre PCA ?

In [21]:
print(f"Ensemble, ces {n_componants} composantes captent {sum(model.explainedVariance)*100:.2f} % de la variance.")

Ensemble, ces 20 composantes captent 91.78 % de la variance.


Affichage de la structure du dataframe *features_df* après la PCA : 

In [22]:
features_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- pcaFeatures: vector (nullable = true)



## **4. Écriture des résultats sous forme de fichiers**

In [23]:
print(f"Les résultats iront ici :\n{PATH_Result}")

Les résultats iront ici :
/mnt/c/Users/Data Science/Desktop/OpenClassrooms/Projets/08 - Réalisez un traitement dans un environnement Big Data sur le Cloud/Projet 08 - Fichiers/data/Results


Enregistrement des données traitées au format "**parquet**" :

In [24]:
features_df.write.mode("overwrite").parquet(PATH_Result)

1/1 [==============================] - 1s 501ms/step===========>  (19 + 1) / 20]


## **5. Chargement des données enregistrées, validation des résultats et export des composantes PCA dans un fichier CSV**

### **5.1 Chargement des données**

On charge les données qui ont été enregistrées au format parquet dans un **DataFrame Pandas** :

In [25]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

On ne va garder que les noms de fichiers au lieu de tout le chemin : 

In [26]:
# Séparer les éléments de 'path' et ne garder que le dernier (nom du fichier)
df['path'] = df['path'].apply(lambda x: x.split('/')[-1])
# Renommer la colonne 'path' en 'filename'
df = df.rename(columns={'path': 'filename'})

Ici les valeurs prennent la forme d'un dictionnaire, car *features* avait été converti en vecteurs spark

In [27]:
df['features'][0]

{'type': 1,
 'size': None,
 'indices': None,
 'values': array([0.        , 0.        , 0.01413861, ..., 0.        , 0.00701549,
        0.        ])}

Nous souhaitons seulement avoir les valeurs : 

In [28]:
df['features'] = df['features'].apply(lambda x: x['values'] if x is not None else None)
df['pcaFeatures'] = df['pcaFeatures'].apply(lambda x: x['values'] if x is not None else None)

### **5.2 Validation des résultats**

On valide que la dimension du vecteur de caractéristiques ('features') des images est bien de dimension 1280, c'est-à-dire la dimension de l'avant dernière couche du modèle *MobileNetV2* :

In [29]:
df.loc[0,'features'].shape

(1280,)

On valide dimension du vecteur de caractéristiques après PCA ('pcaFeatures'), qui doit correspondre au nombre choisi de composantes lors de la PCA : 

In [30]:
df.loc[0,'pcaFeatures'].shape

(20,)

Contenu de notre dataframe Pandas : 

In [31]:
df.head()

,filename,label,features,pcaFeatures
0,r_22_100.jpg,Apple Crimson Snow,"[0.0, 0.0, 0.014138607308268547, 0.0, 0.0, 0.0...","[-1.0369816809897343, -4.820961584087551, 0.52..."
1,r_41_100.jpg,Apple Crimson Snow,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.2477071285247803, ...","[3.746110197763563, -4.938794297068702, -1.596..."
2,r_35_100.jpg,Apple Braeburn,"[0.6380921602249146, 0.035582467913627625, 0.0...","[-12.982453623476486, -5.1146858555668215, 1.6..."
3,r_42_100.jpg,Apple Braeburn,"[0.8061074018478394, 0.07867594808340073, 0.0,...","[-14.861696595808477, -6.190383556400285, 3.36..."
4,r_113_100.jpg,Apple Crimson Snow,"[0.015278063714504242, 0.07300541549921036, 0....","[-11.106028177881674, -1.1916776058719598, -0...."


### **5.3 Export des composantes PCA dans un ficheir CSV**

Enregistrement des pcaFeatures (associées aux noms de fichier et labels) au format CSV : 

In [32]:
# np.printoptions pour éviter l'insertion de "\n" dans 'pcaFeatures' dans notre fichier csv
with np.printoptions(linewidth=10000):
    df[['filename', 'label', 'pcaFeatures']].to_csv(PATH_Result+'/'+'pcaFeatures.csv', index=False, sep='\t')

### **Mesure de la durée d'exécution du notebook**

In [33]:
end_time = time.perf_counter()
elapsed_time = end_time - start_time

# Conversion en minutes et secondes
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"Durée d'exécution du notebook (hors imports) : {minutes:02}:{seconds:02}")

Durée d'exécution du notebook (hors imports) : 03:08
